# 5. Map results
Plot the coordinates we obtained from geocoding into a `folium` map.

In [ ]:
import pandas as pd
import folium
import os

### Paths

In [ ]:
df_results = pd.read_csv(os.path.join('results', 'locations_geocoded.tsv'), sep='\t', quotechar='"', encoding='utf-8')
df_results

In [ ]:
df_geocoded = df_results[pd.to_numeric(df_results.geocode_lat, errors='coerce').notnull()].copy()
df_geocoded['lat_float'] = df_geocoded['geocode_lat'].astype(float)
df_geocoded['lon_float'] = df_geocoded['geocode_lon'].astype(float)

### Mapping functions

In [ ]:
# function to show a text value for the geocode result, if present
def geocode_eval_to_text(geocode_eval):
    if geocode_eval == '1.0':
        return 'correct'
    elif geocode_eval == '0.0':
        return 'incorrect'
    elif geocode_eval == 'nan':
        return 'upstream error'
    else:
        return geocode_eval

# function to decide what goes in a map popup
def to_html_string(row):
    html = "geocoded string: " + str(row['clean_content_loc']) + "<br>"
    html += "result string: " + str(row['geocode_str']) + "<br>"
    html += "result point: " + str(row['geocode_lat']) + ", " + str(row['geocode_lon']) + "<br>"
    html += "result type: " + str(row['geocode_type']) + "<br>"
    #html += "geoparse eval: " + str(row['eval_geoparse']) + "<br>"
    #if str(row['error_analysis']) != 'nan':
    #    html += "error: " + str(row['error_analysis']) + "<br>"
    #html += "geocode eval: " + geocode_eval_to_text(str(row['eval_geocode'])) + "<br>"
    #html += "full pipeline eval: " + str(row['eval_full_pipeline'])
    return html

# show the TP/FP in different colors, if evaluated
def get_color(evaluation):
    if evaluation == 'TP':
        # blue
        return '#3186cc'
    elif evaluation == 'FP':
        # red
        return '#cc0000'
    else:
        # we had a different value than TP or FP
        return '#cc0000'

### Results map

In [ ]:
country_geojson_url = 'https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_admin_0_countries.geojson'

result_map = folium.Map(tiles=None, location=[30, 0], zoom_start=2)
folium.GeoJson(
    country_geojson_url,
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    }
).add_to(result_map)
result_map

for index, row in df_geocoded.iterrows():
    lat = row['lat_float']
    lon = row['lon_float']
    html_for_popup = folium.Html(to_html_string(row), script=True)
    html_popup = folium.Popup(html_for_popup, max_width=400)
    folium.CircleMarker([lat,lon], 
                    radius=5,
                    popup=html_popup,
                    color=get_color('TP'),  # unevaluated
                    fill_color=get_color('TP'),  # unevaluated
                   ).add_to(result_map)
result_map

In [ ]:
# export map to an html file!
result_map.save(os.path.join('maps', 'demo_result_map.html'))